In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
# from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.remote.webelement import WebElement


In [14]:
def scrape_img():
    pass

In [15]:
def scrape_location():
    pass

In [16]:
def scrape_cur_restaurant(restaurant:Restaurant) -> None:
    # scrape ...

    # scrape location

    # scrape image path
    pass

In [17]:
def scrape_restaurants_by_province(province: str, wongnai_regionId:str) -> pd.DataFrame:
    res_restaurant_df = pd.DataFrame()
    cur_query_url = "https://www.wongnai.com/restaurants?categoryGroupId=9&regions=%s" % (wongnai_regionId)
   
    # create webdriver instance using option to mazimize current window
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(cur_query_url)
    
    # find first group of restaurant on the first page
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'dibyTT')))
    all_restaurants_card = driver.find_elements(By.CLASS_NAME, 'dibyTT')

    print(type(all_restaurants_card[0]))
    print(all_restaurants_card[0].text)
    # iterate to scrape each restaurant and set propeties of 'Restaurant' object
    for cur_restaurant_card in all_restaurants_card:
        cur_restaurant = Restaurant()

        # find restaurant name
        cur_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'Dtkmv').text
        cur_sub_name = ''
        # cut substring if there is sub name
        try:
            cur_sub_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'dqdias').text
            print("sub name -> ", cur_sub_name)
            cur_Idx_sub_name = cur_name.rfind(cur_sub_name)
            cur_name = cur_name[:cur_Idx_sub_name]
            
        except Exception as e:
            pass

        print('name -> ', cur_name)
        cur_restaurant.set_name(cur_name)
        cur_restaurant.set_sub_name(cur_sub_name)

        # find restaurant tags
        cur_restaurant_tags = []
        cur_restaurant_tags_elements = cur_restaurant_card.find_elements(By.CLASS_NAME, 'okmRN')
        for cur_element in cur_restaurant_tags_elements:
            cur_restaurant_tags.append(cur_element.text)
        print("cur tags")
        print(cur_restaurant_tags)
        cur_restaurant.set_restaurantTag(cur_restaurant_tags)

        # find rating

        # set some properties of 'Restaurant' object
        # cur_restaurant.set_name()
        # cur_restaurant.set_restaurantTag()
        # cur_restaurant.set_rating(
        #     score=,
        #     rating_count=,
        # )

        # continue click card to scrap more data
        scrape_cur_restaurant(restaurant=cur_restaurant)


    # close current browser tab    
    driver.close() 

    return res_restaurant_df.copy()


In [18]:
cnt_for_drbug = 0
for cur_region_data in ALL_REGION_ID_WONGNAI:
    if(cnt_for_drbug == 1):
        break
    
    cur_province = cur_region_data[1]
    cur_regionId = cur_region_data[2]

    # get dataframe result of all restaurants in current province
    cur_res_allRestaurants_df = scrape_restaurants_by_province(
        wongnai_regionId = cur_regionId,
        province = cur_province
    )
    cnt_for_drbug += 1
    # save result dataframe to .csv
    

<class 'selenium.webdriver.remote.webelement.WebElement'>
+31
Ad ·The Beach Phuket
บีชคลับสไตล์บาหลี ติดทะเล ชมพระอาทิตย์ตก เมนูครบครัน เหมาะกับครอบครัว
฿฿
กึ่งผับ/ร้านเหล้า/บาร์, ผับ/เที่ยวกลางคืน, อาหารนานาชาติ
name ->  The Beach Phuket
cur tags
['กึ่งผับ/ร้านเหล้า/บาร์', 'ผับ/เที่ยวกลางคืน', 'อาหารนานาชาติ']
sub name ->  Phuket
name ->  Wood x Good Cafe คาเฟ่ภูเก็ต
cur tags
['เบเกอรี/เค้ก', 'คาเฟ่']
sub name ->  Patong
name ->  Napa Massage
cur tags
['คาเฟ่']
sub name ->  สามกอง ภูเก็ต
name ->  Pizza Hut
cur tags
['พิซซ่า', 'ฟาสต์ฟู้ด/จานด่วน']
sub name ->  ภูเก็ต
name ->  โกเบนซ์ภูเก็ต
cur tags
['อื่นๆ']
sub name ->  มาร์เก็ตวิลเลจ ภูเก็ต
name ->  Pizza Hut
cur tags
['พิซซ่า', 'ฟาสต์ฟู้ด/จานด่วน']
sub name ->  Phuket Floresta
name ->  Tops Eatery
cur tags
['อาหารนานาชาติ']
sub name ->  Central Patong Phuket
name ->  Tops Eatery
cur tags
['อาหารนานาชาติ']
sub name ->  Porto de Phuket
name ->  Tops Eatery
cur tags
['อาหารนานาชาติ']
sub name ->  -
name ->  Tiger Kola
cur tags
['อาหารต